# Conversational
---

Este tutorial demuestra cómo utilizar un agente optimizado para la conversación. Otros agentes suelen estar optimizados para utilizar herramientas que les permitan encontrar la mejor respuesta, lo que no es ideal en un entorno conversacional en el que es posible que desee que el agente también pueda chatear con el usuario.

Esto se logra con un tipo específico de agente (`conversational-react-description`) que espera usarse con un componente de memoria.


In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

load_dotenv(find_dotenv())

True

Creamos la herramienta para ayudar al agente a buscar informacion en internet

In [33]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the worlds"
    )
]

Configuramos la memoria, para este caso usaremos `ConversationBufferMemory`

In [3]:
# memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

In [4]:
# llm
llm = OpenAI(temperature=0)

In [5]:
# agent chain
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

In [6]:
agent_chain.run(input="hi, i am bob")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Bob, nice to meet you! How can I help you today?

> Finished chain.


'Hi Bob, nice to meet you! How can I help you today?'

In [7]:
agent_chain.run(input="what's my name?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Your name is Bob!

> Finished chain.


'Your name is Bob!'

In [8]:
agent_chain.run("what are some good dinners to make this week, if i like thai food?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: Thai food dinner recipes
Observation: ['70 easy Thai recipes for any night of the week · Thai chicken meatballs · Chiang Mai-style pork larb · Thai curry noodle soup · Thai yellow cauliflower, snake bean ...']
Thought: Do I need to use a tool? No
AI: Here are some great Thai dinner recipes you can try this week: Thai chicken meatballs, Chiang Mai-style pork larb, Thai curry noodle soup, Thai yellow cauliflower, snake bean, and more!

> Finished chain.


'Here are some great Thai dinner recipes you can try this week: Thai chicken meatballs, Chiang Mai-style pork larb, Thai curry noodle soup, Thai yellow cauliflower, snake bean, and more!'

In [9]:
agent_chain.run(input="tell me the last letter in my name, and also tell me who won the world cup in 1978?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: Who won the World Cup in 1978
Observation: Argentina national football team
Thought: Do I need to use a tool? No
AI: The last letter in your name is 'b', and the Argentina national football team won the World Cup in 1978.

> Finished chain.


"The last letter in your name is 'b', and the Argentina national football team won the World Cup in 1978."

In [10]:
agent_chain.run(input="whats the current temperature in pomfret?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: Current temperature in Pomfret
Observation: {'type': 'weather_result', 'temperature': '68', 'unit': 'Fahrenheit', 'precipitation': '10%', 'humidity': '76%', 'wind': '9 mph', 'location': 'Pomfret, CT', 'date': 'Tuesday', 'weather': 'Clear'}
Thought: Do I need to use a tool? No
AI: The current temperature in Pomfret is 68 degrees Fahrenheit, with 10% precipitation, 76% humidity, and 9 mph wind. The weather is currently clear.

> Finished chain.


'The current temperature in Pomfret is 68 degrees Fahrenheit, with 10% precipitation, 76% humidity, and 9 mph wind. The weather is currently clear.'

# Using a chat model
---

El tipo de agente chat-conversational-react-description nos permite crear un agente conversacional utilizando un modelo de chat en lugar de un LLM


In [29]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# llm
llm = ChatOpenAI(
    temperature=0,
)

# memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

# agent chain
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,   
)

In [30]:
agent_chain.run(input="hi, i am bob")



> Entering new AgentExecutor chain...


{
    "action": "Final Answer",
    "action_input": "Hello Bob! How can I assist you today?"
}

> Finished chain.


'Hello Bob! How can I assist you today?'

In [31]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi, i am bob', additional_kwargs={}, example=False),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, example=False)]}

In [32]:
agent_chain.run(input="what's my name?")



> Entering new AgentExecutor chain...
Could not parse LLM output: Your name is Bob.
Observation: Invalid or incomplete response
Thought:{
    "action": "Final Answer",
    "action_input": "Your name is Bob."
}

> Finished chain.


'Your name is Bob.'